In [4]:
import numpy as np
import pandas as pd
import re

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
#nltk.download()

## 处理训练集

In [2]:
def data_preprocessing(data_path, label_path):
    #labels
    labels = []
    with open(label_path, 'r', encoding='utf-8') as trainLabel:
        for label in trainLabel:
            labels.append(int(re.sub(r'\n', '', label)))
    labels = np.array(labels)
    
    sentences = []
    with open(data_path, 'r', encoding='utf-8') as trainData:
        for sentence in trainData:
            sentences.append(re.sub(r'(<br /><br />)|(-.*?-)', ' ', sentence))
    
    #sentences
    stemmer = SnowballStemmer("english")
    for i in range(len(sentences)):
        sentences[i] = re.sub(r'["\')\(\*\.,!?``:;]| \n', '', sentences[i]).split(' ')
        #变为小写,词形还原
        for j, word in enumerate(sentences[i]):
            sentences[i][j] = stemmer.stem(word.lower())
    
    #除去停用词
    stop_word_list2 = []
    with open('data/stop_word2.txt', 'r', encoding='utf-8') as st:
        stop_word_list2 = st.read().split('\n')
        
    for sentence in sentences:
        for word in sentence.copy():
            if word in stop_word_list2:
                sentence.remove(word)
    return sentences, labels

In [6]:
sentences, labels = data_preprocessing('data/5/trainData.txt', 'data/5/trainLabel.txt')

### 生成新的数据

In [49]:
import pickle
with open('data/5/5_new_train.pkl', 'wb') as nt:
    pickle.dump(sentences, nt)

### 读取数据

In [50]:
with open('data/5/5_new_train.pkl', 'rb') as nt:
    sentences = pickle.load(nt)

### 尝试word2vec

In [13]:
import gensim

In [14]:
model = gensim.models.Word2Vec(sentences, min_count=1)

In [59]:
for i in range(100):
    print('similarity:',model.n_similarity(sentences[0], sentences[i]))
    print('label:', labels[0], labels[i])

similarity: 1.0
label: 2 2
similarity: 0.8362449
label: 2 3
similarity: 0.75267655
label: 2 0
similarity: 0.7636352
label: 2 4
similarity: 0.44443733
label: 2 2
similarity: 0.8459352
label: 2 4
similarity: 0.6981785
label: 2 4
similarity: 0.76213104
label: 2 2
similarity: 0.81456167
label: 2 1
similarity: 0.5252778
label: 2 3
similarity: 0.54591596
label: 2 0
similarity: 0.8151882
label: 2 4
similarity: 0.6986904
label: 2 3
similarity: 0.6273449
label: 2 3
similarity: 0.8459405
label: 2 4
similarity: 0.6300418
label: 2 4
similarity: 0.71485186
label: 2 3
similarity: 0.77421886
label: 2 4
similarity: 0.6536436
label: 2 3
similarity: 0.7176421
label: 2 3
similarity: 0.71228427
label: 2 1
similarity: 0.7901938
label: 2 0
similarity: 0.6827219
label: 2 1
similarity: 0.7213849
label: 2 3
similarity: 0.6457514
label: 2 4
similarity: 0.60576767
label: 2 3
similarity: 0.79708546
label: 2 3
similarity: 0.67560965
label: 2 2
similarity: 0.8136377
label: 2 3
similarity: 0.59769946
label: 2 4
simi

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  
/home/pp/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## 提取所有单词

In [38]:
all_word = []
for sentence in sentences:
    for word in sentence:
        if word not in all_word:
            all_word.append(word)
all_word

['watch',
 'movi',
 'netflix',
 'disc',
 'mom',
 'enjoy',
 'thriller',
 'agent',
 'dure',
 '60s',
 'tri',
 'bring',
 'trial',
 'nazi',
 'war',
 'crimin',
 'gynecologist',
 'woman',
 'rachel',
 'jessica',
 'chastain',
 'pose',
 'patient',
 'complic',
 'ensu',
 'result',
 'anoth',
 'timelin',
 '30',
 'helen',
 'mirren',
 'play',
 'guilt',
 'fellow',
 'deal',
 'haunt',
 'quit',
 'entertain',
 'especi',
 'scene',
 'ms',
 'doctor',
 'talk',
 'befor',
 'debt',
 'recommend',
 'figur',
 'larger',
 'life',
 'edgar',
 'hoover',
 'rank',
 'career',
 'span',
 '55',
 '1917',
 'death',
 '1972',
 'age',
 '77',
 'leonardo',
 'dicaprio',
 'perform',
 'lifetim',
 'bull',
 'dog',
 'forg',
 'sleepi',
 'feder',
 'agenc',
 'premier',
 'law',
 'enforc',
 'master',
 'art',
 'scientif',
 'detect',
 'onli',
 'actor',
 'jimmi',
 'cagney',
 'lrb',
 'rrb',
 'assum',
 'air',
 'profession',
 'dissect',
 'crime',
 'arm',
 'time',
 'hero',
 'villain',
 'express',
 'reserv',
 'investig',
 'nt',
 'idea',
 'summarili',
 

## 生成矩阵

In [40]:
#计算TF矩阵
tf = np.zeros((len(sentences), len(all_word)), dtype=bool) #(24000,38710)
for i, sentence in enumerate(sentences):
    for word in sentence:
        index = all_word.index(word)
        tf[i][index] = 1

##### 对给定词进行卡方检验

In [47]:
def calc_chi_square(i, labels):
    tf_i = tf[:, i]
    pos, neg = tf_i[labels==1], tf_i[labels==0]
    A = pos.sum() #包含特征且为正样本
    B = neg.sum() #包含特征且为负样本
    C = len(pos) - A #不包含特征且为正样本
    D = len(neg) - B #不包含特征且为负样本
    print(A,B,C,D)
    if A+B <= 50: #减少低频性缺陷
        return 0 
    kf = (A * D - B * C) ** 2 / ((A+B)*(C+D))
    return kf

In [63]:
import queue
Q = queue.PriorityQueue()
for i in range(len(all_word)):
    Q.put((-calc_chi_square(i, labels), i)) #取负号，小项堆排序

1310 1321 3369 3396
2786 2798 1893 1919
5 7 4674 4710
23 10 4656 4707
48 61 4631 4656
939 803 3740 3914
282 301 4397 4416
135 132 4544 4585
371 343 4308 4374
27 15 4652 4702
872 929 3807 3788
377 341 4302 4376
31 24 4648 4693
55 35 4624 4682
357 287 4322 4430
95 80 4584 4637
3 0 4676 4717
396 368 4283 4349
56 49 4623 4668
57 44 4622 4673
5 8 4674 4709
34 30 4645 4687
45 49 4634 4668
81 61 4598 4656
33 26 4646 4691
219 234 4460 4483
714 694 3965 4023
3 4 4676 4713
84 83 4595 4634
38 36 4641 4681
7 10 4672 4707
1360 1330 3319 3387
38 21 4641 4696
75 61 4604 4656
307 226 4372 4491
106 96 4573 4621
885 807 3794 3910
771 761 3908 3956
490 502 4189 4215
1273 1291 3406 3426
34 29 4645 4688
117 126 4562 4591
321 298 4358 4419
686 734 3993 3983
23 20 4656 4697
420 302 4259 4415
200 200 4479 4517
26 28 4653 4689
944 759 3735 3958
32 33 4647 4684
13 15 4666 4702
45 42 4634 4675
198 179 4481 4538
20 14 4659 4703
4 21 4675 4696
1 0 4678 4717
368 366 4311 4351
15 15 4664 4702
322 256 4357 4461
4 4 4

49 39 4630 4678
72 63 4607 4654
0 1 4679 4716
140 128 4539 4589
2 0 4677 4717
17 18 4662 4699
208 207 4471 4510
81 72 4598 4645
58 50 4621 4667
179 200 4500 4517
3 7 4676 4710
820 806 3859 3911
1 5 4678 4712
120 123 4559 4594
397 505 4282 4212
72 50 4607 4667
196 239 4483 4478
717 769 3962 3948
85 148 4594 4569
290 406 4389 4311
4 5 4675 4712
8 5 4671 4712
324 369 4355 4348
260 297 4419 4420
42 53 4637 4664
31 26 4648 4691
28 14 4651 4703
29 18 4650 4699
262 315 4417 4402
4 2 4675 4715
83 46 4596 4671
28 24 4651 4693
60 55 4619 4662
183 148 4496 4569
5 4 4674 4713
139 118 4540 4599
1 1 4678 4716
23 24 4656 4693
5 3 4674 4714
107 95 4572 4622
22 19 4657 4698
107 92 4572 4625
235 208 4444 4509
652 631 4027 4086
10 5 4669 4712
5 7 4674 4710
2 0 4677 4717
127 132 4552 4585
28 26 4651 4691
60 40 4619 4677
19 20 4660 4697
31 22 4648 4695
87 85 4592 4632
67 53 4612 4664
74 81 4605 4636
21 9 4658 4708
94 116 4585 4601
102 95 4577 4622
362 350 4317 4367
42 34 4637 4683
580 545 4099 4172
54 54 4

0 1 4679 4716
57 39 4622 4678
72 68 4607 4649
87 100 4592 4617
0 1 4679 4716
7 5 4672 4712
7 3 4672 4714
98 104 4581 4613
20 14 4659 4703
2 2 4677 4715
107 75 4572 4642
96 49 4583 4668
0 0 4679 4717
108 111 4571 4606
24 21 4655 4696
42 33 4637 4684
30 47 4649 4670
118 107 4561 4610
8 8 4671 4709
10 5 4669 4712
4 2 4675 4715
6 7 4673 4710
127 95 4552 4622
35 35 4644 4682
98 89 4581 4628
32 27 4647 4690
25 24 4654 4693
9 9 4670 4708
112 122 4567 4595
19 28 4660 4689
354 373 4325 4344
15 23 4664 4694
49 51 4630 4666
24 16 4655 4701
62 74 4617 4643
33 46 4646 4671
39 36 4640 4681
167 168 4512 4549
39 70 4640 4647
9 9 4670 4708
268 324 4411 4393
56 74 4623 4643
22 25 4657 4692
87 85 4592 4632
88 106 4591 4611
210 196 4469 4521
41 48 4638 4669
78 71 4601 4646
11 10 4668 4707
21 23 4658 4694
12 16 4667 4701
158 133 4521 4584
2 8 4677 4709
30 43 4649 4674
160 161 4519 4556
610 510 4069 4207
90 95 4589 4622
8 9 4671 4708
87 82 4592 4635
0 0 4679 4717
2 1 4677 4716
161 159 4518 4558
58 50 4621 4

35 39 4644 4678
10 2 4669 4715
12 10 4667 4707
15 11 4664 4706
1 1 4678 4716
6 11 4673 4706
7 8 4672 4709
1 0 4678 4717
1 0 4678 4717
0 4 4679 4713
0 0 4679 4717
124 96 4555 4621
16 15 4663 4702
62 56 4617 4661
11 14 4668 4703
104 98 4575 4619
18 12 4661 4705
74 76 4605 4641
18 9 4661 4708
34 27 4645 4690
69 81 4610 4636
47 57 4632 4660
1 2 4678 4715
36 44 4643 4673
4 5 4675 4712
2 2 4677 4715
55 45 4624 4672
67 49 4612 4668
25 15 4654 4702
0 2 4679 4715
18 13 4661 4704
109 100 4570 4617
181 164 4498 4553
2 2 4677 4715
18 24 4661 4693
2 0 4677 4717
12 11 4667 4706
66 48 4613 4669
1 5 4678 4712
27 17 4652 4700
7 3 4672 4714
30 29 4649 4688
0 3 4679 4714
2 1 4677 4716
106 96 4573 4621
14 17 4665 4700
42 32 4637 4685
3 5 4676 4712
46 58 4633 4659
20 10 4659 4707
35 26 4644 4691
1 1 4678 4716
1 1 4678 4716
81 73 4598 4644
12 8 4667 4709
12 6 4667 4711
39 34 4640 4683
91 72 4588 4645
193 177 4486 4540
184 155 4495 4562
7 13 4672 4704
39 24 4640 4693
0 0 4679 4717
4 4 4675 4713
160 117 4519 

68 54 4611 4663
88 60 4591 4657
48 40 4631 4677
4 1 4675 4716
15 16 4664 4701
1 1 4678 4716
6 2 4673 4715
1 1 4678 4716
1 0 4678 4717
7 3 4672 4714
7 7 4672 4710
13 15 4666 4702
142 155 4537 4562
56 43 4623 4674
27 37 4652 4680
20 25 4659 4692
26 34 4653 4683
23 29 4656 4688
30 35 4649 4682
34 26 4645 4691
20 16 4659 4701
42 36 4637 4681
78 49 4601 4668
79 66 4600 4651
9 5 4670 4712
57 47 4622 4670
50 50 4629 4667
47 35 4632 4682
8 2 4671 4715
37 33 4642 4684
20 14 4659 4703
22 15 4657 4702
25 18 4654 4699
4 10 4675 4707
0 1 4679 4716
4 4 4675 4713
11 6 4668 4711
10 13 4669 4704
64 50 4615 4667
6 1 4673 4716
14 17 4665 4700
5 2 4674 4715
3 4 4676 4713
22 25 4657 4692
15 20 4664 4697
19 10 4660 4707
15 15 4664 4702
4 1 4675 4716
10 8 4669 4709
60 51 4619 4666
5 2 4674 4715
68 64 4611 4653
22 16 4657 4701
191 207 4488 4510
5 2 4674 4715
3 1 4676 4716
33 36 4646 4681
3 2 4676 4715
15 19 4664 4698
25 26 4654 4691
22 19 4657 4698
7 10 4672 4707
33 30 4646 4687
99 141 4580 4576
4 7 4675 4710

...

0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
1 2 4678 4715
0 1 4679 4716
0 2 4679 4715
0 0 4679 4717
0 1 4679 4716
1 1 4678 4716
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 2 4679 4715
0 1 4679 4716
1 0 4678 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
1 1 4678 4716
2 0 4677 4717
2 0 4677 4717
2 0 4677 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
2 0 4677 4717
1 0 4678 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 2 46

0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 2 4679 4715
0 1 4679 4716
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
2 1 4677 4716
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
0 1 4679 4716
0 3 4679 4714
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
1 0 4678 4717
0 1 4679 4716
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 46

0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
1 0 4678 4717
2 0 4677 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
0 2 4679 4715
1 0 4678 4717
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 2 4679 4715
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
0 1 4679 4716
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 46

0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 1 4679 4716
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
1 0 4678 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
2 0 4677 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 0 46

0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 1 4679 4716
0 1 4679 4716
0 0 4679 4717
2 0 4677 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
1 0 4678 4717
1 0 4678 4717
0 0 4679 4717
1 0 4678 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 1 4679 4716
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
0 0 4679 4717
1 0 4678 4717
0 0 46

In [64]:
all_word_by_kf = []
for i in range(10000):
    top = Q.get()
    if top[0] == 0: break
    all_word_by_kf.append(all_word[top[1]])

In [65]:
len(all_word_by_kf)

2542

## SVM

In [66]:
#编码所有训练样本
train_one_hot = np.zeros((len(sentences), len(all_word_by_kf)), dtype='int8')
for i, sentence in enumerate(sentences):
    for word in sentence:
        try:
            train_one_hot[i][all_word_by_kf.index(word)] += 1
        except:
            train_one_hot[-1] += 1

In [67]:
train_length = 20000
train_data = train_one_hot[:train_length]
train_labels = labels[:train_length]
validation_data = train_one_hot[train_length:]
validation_labels = labels[train_length:]

In [68]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_data, train_labels)

/home/pp/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [77]:
result = clf.predict(validation_data)

In [78]:
accuracy = 0
for i in range(len(result)):
    if result[i] == validation_labels[i]:
        accuracy += 1
accuracy /= len(result)

In [79]:
accuracy

0.34425

## PLA

In [81]:
from sklearn.linear_model import Perceptron

In [ ]:
PLA = 